In [ ]:
!pip install transformers
!pip install accelerate
!pip install accelerate --upgrade
!pip install transformers --upgrade

!pip install transformers[torch]
!pip install transformers[torch] -U

import accelerate
import transformers
print("Accelerate version:", accelerate.__version__)
print("Transformers version:", transformers.__version__)

Accelerate version: 0.30.0
Transformers version: 4.40.1


In [ ]:
import torch
import torch.nn as nn

from transformers import GPT2TokenizerFast, GPT2LMHeadModel
from tqdm.auto import tqdm

import pandas as pd
import numpy as np

In [ ]:
from transformers import Trainer
from transformers import TrainingArguments

model_save_path = '/content/drive/MyDrive/CS271_Project/FoodGPT'
args = TrainingArguments( output_dir = model_save_path,
                         per_device_train_batch_size=2,
                         per_device_eval_batch_size=2,
                         gradient_accumulation_steps=2,
                         report_to='none',
                         num_train_epochs=1,
                         save_strategy='no'
                        )

In [ ]:
model_name = 'gpt2'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model_save_path = '/content/drive/MyDrive/CS271_Project/FoodGPT'

In [ ]:
tokenizer = GPT2TokenizerFast.from_pretrained(model_name,
                                              bos_token='<|startoftext|>',
                                              eos_token='<|endoftext|>',
                                              unk_token='<|unknown|>',
                                              pad_token='<|pad|>'
                                             )
model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Embedding(50260, 768)

In [ ]:
tokenizer.save_pretrained(model_save_path)

('/content/drive/MyDrive/CS271_Project/FoodGPT/tokenizer_config.json',
 '/content/drive/MyDrive/CS271_Project/FoodGPT/special_tokens_map.json',
 '/content/drive/MyDrive/CS271_Project/FoodGPT/vocab.json',
 '/content/drive/MyDrive/CS271_Project/FoodGPT/merges.txt',
 '/content/drive/MyDrive/CS271_Project/FoodGPT/added_tokens.json',
 '/content/drive/MyDrive/CS271_Project/FoodGPT/tokenizer.json')

In [ ]:
tokenizer.convert_tokens_to_ids(['<|pad|>'])

[50259]

In [ ]:
def generate(prompt):
    inputs = tokenizer.encode_plus(prompt, return_tensors='pt')
    output = model.generate(**inputs,max_length=256,do_sample=True,pad_token_id=50259)
    print(tokenizer.decode(output[0]))

In [ ]:
tokenizer.special_tokens_map

{'bos_token': '<|startoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|unknown|>',
 'pad_token': '<|pad|>'}

In [ ]:
tokenizer.convert_tokens_to_ids(['<|startoftext|>'],)

[50257]

In [ ]:
My_data = pd.read_csv('/content/drive/MyDrive/CS271_Project/recipes_w_search_terms.csv')
My_data = My_data.sample(frac=1)
My_data.reset_index(drop=True,inplace=True)

In [ ]:
My_data.head()

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms
0,31377,"RED, WHITE and BLUE Frozen Dessert",This is a great dessert for patriotic holidays...,"['sweetened condensed milk', 'lemon juice', 'l...","[""1 (14 ounce) can sweetened condensed mil...",1 (101 g),16,"['In a large bowl, combine condensed milk, lem...","['30-minutes-or-less', 'time-to-make', 'course...","{'dessert', 'dinner', 'low-sodium'}"
1,337404,Cranberry Chutney - Tyler Florence,"From ""Tyler's Ultimate,"" episode ""Ultimate Cor...","['frozen cranberries', 'red wine', 'white sugar']","[""12 ounces frozen cranberries (2 cups)"",...",1 (132 g),4,"['Combine berries, wine and sugar in a small s...","['30-minutes-or-less', 'time-to-make', 'course...","{'low-calorie', 'low-sodium', 'vegetarian', 'h..."
2,78853,The Best One-Bowl Hash Brown Potato Casserole,My family loves this and I make it on regular ...,"['cream of mushroom soup', 'milk', 'sour cream...","[""1 (10 ounce) can undiluted cream of mushr...",1 (253 g),8,"['Set oven to 350°F.', 'Butter a 13x9-inch cas...","['60-minutes-or-less', 'time-to-make', 'course...","{'side', 'casserole', 'dinner'}"
3,317162,Herb Rubbed Grilled Scallops,Mediterranean flavors,"['dried tarragon', 'dried thyme leaves', 'drie...","[""1 teaspoon dried tarragon"",""1/2 teaspo...",1 (119 g),4,"['In a shallow dish, cover eight 10-inch bambo...","['time-to-make', 'course', 'main-ingredient', ...","{'low-calorie', 'healthy', 'low-carb', 'appeti..."
4,290373,Mexican Side Salad,A quick d simple salad to serve with your Mexi...,"['romaine lettuce', 'tomatoes', 'green onions'...","[""1 romaine lettuce, torn into bite size...",1 (298 g),4,"['Wash and prepare veggies.', 'In blender pure...","['15-minutes-or-less', 'time-to-make', 'course...","{'mexican', 'side', 'vegetarian', 'salad'}"


In [ ]:
My_data.tags.value_counts()

tags
['60-minutes-or-less', 'time-to-make', 'preparation']                                                                                                                                                                                                                                                                                             626
['time-to-make', 'course', 'preparation', 'desserts', 'cakes', '4-hours-or-less']                                                                                                                                                                                                                                                                 621
['60-minutes-or-less', 'time-to-make', 'course', 'preparation', 'desserts', 'cakes']                                                                                                                                                                                                                                   

In [ ]:
My_data.info(verbose = False, memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494963 entries, 0 to 494962
Columns: 10 entries, id to search_terms
dtypes: int64(2), object(8)
memory usage: 980.9 MB


In [ ]:
print(My_data.shape[0])

494963


In [ ]:
filtered_df = My_data[My_data['tags'].apply(lambda x: '30-minutes-or-less' in x)]

print(filtered_df.shape[0])
# print(filtered_df)

116312


In [ ]:
df = filtered_df[["ingredients_raw_str", "steps"]]
df.info(verbose = False, memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
Index: 116312 entries, 0 to 494956
Columns: 2 entries, ingredients_raw_str to steps
dtypes: object(2)
memory usage: 110.1 MB


In [ ]:
df.drop_duplicates(inplace=True)
df.shape[0]
df.head()

<ipython-input-20-e3a7ee77072f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


,ingredients_raw_str,steps
0,"[""1 (14 ounce) can sweetened condensed mil...","['In a large bowl, combine condensed milk, lem..."
1,"[""12 ounces frozen cranberries (2 cups)"",...","['Combine berries, wine and sugar in a small s..."
6,"[""1/4 cup orange juice"",""2 tablespoons ...","['Preheat oven 400 degrees.', 'Coat shallow ba..."
7,"[""16 slices sandwich bread"",""3 tablespo...","['preheat oven to 375.', 'Mix cottage cheese, ..."
8,"[""20 medium shrimp, shelled and deviened ...","['Bring a large pot of water to boil.', 'Skewe..."


In [ ]:
def print_recipe(idx):
    print(f"{df['ingredients_raw_str'][idx]}\n\n{df['steps'][idx]}")

In [ ]:

df.loc[:, 'ingredients_raw_str'] = df['ingredients_raw_str'].str.replace(r"[\[\]']", "", regex=True)

# Use .loc to update the 'steps' column
df.loc[:, 'steps'] = df['steps'].str.replace(r"[\[\]']", "", regex=True)


In [ ]:
def form_string(ingredient,steps):
    s = f"<|startoftext|>Ingredients:\n{ingredient.strip()}\n\nSteps:\n{steps.strip()}<|endoftext|>"
    return s

In [ ]:
data = df.apply(lambda x:form_string(x['ingredients_raw_str'],x['steps']),axis=1).to_list()

NameError: name 'df' is not defined

In [ ]:
train_size = 0.6
train_len = int(train_size * len(data))
train_data = data[:train_len]
val_data = data[train_len:]

In [ ]:
class RecipeDataset:
    def __init__(self,data):
        self.data = data
        self.input_ids = []
        self.attn_masks = []

        for item in tqdm(data):
            encodings = tokenizer.encode_plus(item,
                                              truncation=True,
                                              padding='max_length',
                                              max_length=1024,
                                              return_tensors='pt'
                                             )
            self.input_ids.append(torch.squeeze(encodings['input_ids'],0))
            self.attn_masks.append(torch.squeeze(encodings['attention_mask'],0))

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
def collate_fn(batch):
    return {
        'input_ids': torch.stack([item[0] for item in batch]),
        'attention_mask': torch.stack([item[1] for item in batch]),
        'labels': torch.stack([item[0] for item in batch])
    }

In [ ]:
train_ds = RecipeDataset(train_data)
val_ds = RecipeDataset(val_data)

  0%|          | 0/69753 [00:00<?, ?it/s]

  0%|          | 0/46502 [00:00<?, ?it/s]

In [ ]:
args = TrainingArguments( output_dir = model_save_path,
                         per_device_train_batch_size=2,
                         per_device_eval_batch_size=2,
                         gradient_accumulation_steps=2,
                         report_to='none',
                         num_train_epochs=1,
                         save_strategy='no'
                        )

In [ ]:
optim = torch.optim.AdamW(model.parameters(),lr=5e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optim,20,eta_min=1e-7)

In [ ]:
trainer = Trainer(model,
                  args,
                  train_dataset=train_ds,
                  eval_dataset=val_ds,
                  data_collator=collate_fn,
                  optimizers=(optim,scheduler)
                 )

In [ ]:
trainer.train()


Step,Training Loss
500,0.690600
1000,0.478100
1500,0.465300
2000,0.450000
2500,0.441200
3000,0.438500
3500,0.436600
4000,0.432100
4500,0.430300
5000,0.422700


TrainOutput(global_step=17438, training_loss=0.42165051169427714, metrics={'train_runtime': 24594.7946, 'train_samples_per_second': 2.836, 'train_steps_per_second': 0.709, 'total_flos': 3.6451283632128e+16, 'train_loss': 0.42165051169427714, 'epoch': 0.9999713278091579})

In [ ]:
# Save the model
trainer.save_model('/content/drive/MyDrive/CS271_Project')
# trainer.save_pretrained('/content/drive/MyDrive/CS271_Project')


# Save the tokenizer
tokenizer.save_pretrained('/content/drive/MyDrive/CS271_Project')

('/content/drive/MyDrive/CS271_Project/tokenizer_config.json',
 '/content/drive/MyDrive/CS271_Project/special_tokens_map.json',
 '/content/drive/MyDrive/CS271_Project/vocab.json',
 '/content/drive/MyDrive/CS271_Project/merges.txt',
 '/content/drive/MyDrive/CS271_Project/added_tokens.json',
 '/content/drive/MyDrive/CS271_Project/tokenizer.json')

In [ ]:
# Save the model
trainer.save_model('/content/drive/MyDrive/CS271_Project/mySavedModel')
# trainer.save_pretrained('/content/drive/MyDrive/CS271_Project/')


# Save the tokenizer
tokenizer.save_pretrained('/content/drive/MyDrive/CS271_Project/mySavedModel')

('/content/drive/MyDrive/CS271_Project/mySavedModel/tokenizer_config.json',
 '/content/drive/MyDrive/CS271_Project/mySavedModel/special_tokens_map.json',
 '/content/drive/MyDrive/CS271_Project/mySavedModel/vocab.json',
 '/content/drive/MyDrive/CS271_Project/mySavedModel/merges.txt',
 '/content/drive/MyDrive/CS271_Project/mySavedModel/added_tokens.json',
 '/content/drive/MyDrive/CS271_Project/mySavedModel/tokenizer.json')

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the model
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/CS271_Project/mySavedModel")

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/CS271_Project/mySavedModel")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import pipeline

# Create a text generation pipeline
pl = pipeline("text-generation", model="/content/drive/MyDrive/CS271_Project/mySavedModel")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def create_prompt(ingredients):
    ingredients = ','.join([x.strip().lower() for x in ingredients.split(',')])
    ingredients = ingredients.strip().replace(',','\n')
    s = f"<|startoftext|>Ingredients:\n{ingredients}\n"
    return s

In [ ]:
ingredients = ['Rice,Potatoes,Tomatoes, Spinach','Rice,Potatoes,Tomatoes, Spinach']


In [ ]:
for ing in ingredients:
    prompt = create_prompt(ing)
    print(pl(prompt,
         max_new_tokens=512,
         penalty_alpha=0.6,
         top_k=4,
         pad_token_id=50259
        )[0]['generated_text'])

<|startoftext|>Ingredients:
rice
potatoes
tomatoes
spinach
rice,tomatoes,spinach,rice,tomatoes","rice,tomatoes,spinach,rice,tomatoes,spinach,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes","rice,tomatoes,spinach,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes","rice,tomatoes,spinach,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes,rice,tomatoes